# 04 Temp understand logic

#### Connect to GD

In [ ]:
# set up connection to your google drive
# please click on the link generated and enter the authorisation code

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Colab/statbot

In [ ]:
! pip install rank-bm25

In [ ]:
 # update spacy 
! pip install -U spacy
! python -m spacy info

#### Skript

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("data/datasets_overview.csv")

In [ ]:
import spacy
from rank_bm25 import BM25Okapi
from tqdm import tqdm
nlp = spacy.load("de_core_news_lg")
text_list = df["dataset_title"].str.lower().values
text_list=np.unique(text_list)
tok_text=[] # for our tokenised corpus
#Tokenising using SpaCy:
for doc in tqdm(nlp.pipe(text_list, disable=["tagger", "parser","ner"])):
    tok = [t.text for t in doc if t.is_alpha]
    tok_text.append(tok)

In [ ]:
text_list

In [ ]:
bm25 = BM25Okapi(tok_text)

In [ ]:
query = "Eigenkapital"
tokenized_query = query.lower().split(" ")
import time
t0 = time.time()
results = bm25.get_top_n(tokenized_query, text_list, n=5)
t1 = time.time()
print(f'Searched in {round(t1-t0,3) } seconds \n')

for i in results:
    print(i)